In [1]:
!wget http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/mono/OpenSubtitles.raw.en.gz

--2020-06-25 10:06:32--  http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/mono/OpenSubtitles.raw.en.gz
Resolving opus.nlpl.eu (opus.nlpl.eu)... 193.166.25.9
Connecting to opus.nlpl.eu (opus.nlpl.eu)|193.166.25.9|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/en.txt.gz [following]
--2020-06-25 10:06:32--  https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/en.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3663376519 (3.4G) [application/gzip]
Saving to: ‘download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en.gz’

download.php?f=Open 100%[===================>]   3.41G  22.9MB/s    in 2m 35s  

2020-06-25 10:09:09 (22.5 MB/s) - ‘download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en.gz

In [2]:
!gunzip opensubs.gz

In [3]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-06-25 10:14:18--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-06-25 10:14:19--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-06-25 10:14:19--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [4]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [5]:
import os
import numpy as np

In [6]:
embeds = {}
with open("glove.6B.50d.txt", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeds[word] = vector

In [7]:
print(embeds['nvidia'])

[-0.33102  -0.75616   1.9245    2.4399   -0.089388 -0.032776  0.51466
 -0.50618  -1.4438    0.91152   0.33239   0.36876  -0.18953  -0.76038
 -0.532    -0.68048  -1.7754    1.3991    0.46812  -1.3685    0.68417
 -1.4185   -0.38349  -0.66051   0.090225  0.34747  -0.39275  -0.65303
 -0.10736  -1.195    -0.050039 -0.56808  -0.064523 -0.45444   0.9146
  0.19898  -0.38     -0.018667 -0.81614  -0.7589    0.73566  -0.35776
 -1.1478   -0.63147   0.18396  -0.7315    0.79278   0.46402   0.32069
 -0.050436]


In [8]:
dataset = []
i = 0
with open("opensubs", 'r') as f:
    for line in f:
        values = line.lower()
        dataset.append(values)
        i = i + 1
        if i == 1000000:
          break

In [9]:
vocab_size = 20000
embedding_dim = 200
max_length = 20               #max(len(i.split())  for i in dataset)
trunc_type='post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(dataset)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(dataset)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type, padding = 'post')

In [10]:
i = 5
print(len(word_index))
print(dataset[i])
print(padded[i])

45068
exactly two years ago today, she and i buried a time capsule here.

[ 515  136  234  403  294   69    8    4 1594    6   82 6233   36    0
    0    0    0    0    0    0]


In [11]:
index_word = {}
for word , index in word_index.items():
    index_word[index] = word

In [12]:
input_seq = []
input_labs = []
j = 0
for i in padded:
  if j%2 == 0:
    input_seq.append(i)
  if j%2 == 1:
    input_labs.append(i)
  j+=1
  if j == 20000:
    break

input_seq = np.array(input_seq)
input_labs = np.array(input_labs)

print(input_seq[3])
print(input_labs[3])
print(input_seq.shape)
print(input_labs.shape)

[  27 1024    5  347   36  136  234  350   33   69  839   46  279    0
    0    0    0    0    0    0]
[ 30  87   5 169   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
(10000, 20)
(10000, 20)


In [13]:
# input_seq = []

# for i in range(500):
#   sent_embeds = []
#   temp1 = dataset[2*i].split()
#   for j in temp1:
#     if j in embeds:
#       temp2 = embeds[j]
#     else:
#       temp2 = np.zeros((embedding_dim), dtype=np.float32)
#     sent_embeds.append(temp2)
#   temp3 = np.zeros((embedding_dim), dtype=np.float32)
#   while len(sent_embeds) < max_length:
#     sent_embeds.append(temp3)
#   input_seq.append(sent_embeds)

In [14]:
# input_labs = []

# for i in range(500):
#   sent_embeds = []
#   temp1 = dataset[2*i+1].split()
#   for j in temp1:
#     if j in embeds:
#       temp2 = embeds[j]
#     else:
#       temp2 = np.zeros((embedding_dim), dtype=np.float32)
#     sent_embeds.append(temp2)
#   temp3 = np.zeros((embedding_dim), dtype=np.float32)
#   while len(sent_embeds) < max_length:
#     sent_embeds.append(temp3)
#   input_labs.append(sent_embeds)

In [15]:
# i = 1

# print(input_labs[i])
# print(dataset[2*i+1].split())

# print(embeds['promised'])

# for l in dataset[2*i].split():
#   a = l not in embeds
#   print(a)

In [16]:
# final_input_seq = np.array(input_seq)
# final_input_labs = np.array(input_labs)
# print(final_input_seq.shape)
# print(final_input_seq[1])

In [17]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply, Embedding
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K

Using TensorFlow backend.


In [18]:
def create_word_embeddings(file_path):
    
    with open(file_path , 'r') as f:
        wordToEmbedding = {}
        wordToIndex = {}
        indexToWord = {}
        
        for line in f:
            data = line.strip().split()
            token = data[0]
            wordToEmbedding[token] = np.array(data[1:] ,dtype = np.float64)

        tokens = sorted(wordToEmbedding.keys())
        for idx , token in enumerate(tokens):
            idx = idx + 1 #for zero masking
            wordToIndex[token] = idx
            indexToWord[idx] = token

    return wordToEmbedding , wordToIndex , indexToWord

In [19]:
def create_pretrained_embedding_layer(wordToEmbedding , wordToIndex , indexToWord):
    
    vocablen = 20000 #for zero masking
    embedding_dimensions = 200
    
    embeddingMatrix = np.zeros((vocablen , embedding_dimensions))
    for word , index in word_index.items():
        if word not in wordToEmbedding.keys():
            embeddingMatrix[index ,:] = np.random.uniform(low = 0 , high =1 ,size = (1,200))
            if index == vocablen-1:
              break
        else :
            embeddingMatrix[index , :] = wordToEmbedding[word]
            if index == vocablen-1:
              break
        
    embeddingLayer = Embedding(vocablen , embedding_dimensions , weights = [embeddingMatrix] , trainable = False)
    print(embeddingMatrix.shape)
    
    return embeddingLayer

In [20]:
wordToEmbedding , wordToIndex , indexToWord = create_word_embeddings('glove.6B.200d.txt')

In [21]:
embeddingLayer = create_pretrained_embedding_layer(wordToEmbedding , wordToIndex , indexToWord)

(20000, 200)


In [22]:
print(len(word_index))

45068


In [23]:
def softmax(x, axis=1):
    """Softmax activation function.
    # Arguments
        x : Tensor.
        axis: Integer, axis along which the softmax normalization is applied.
    # Returns
        Tensor, output of softmax transformation.
    # Raises
        ValueError: In case `dim(x) == 1`.
    """
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D')

In [24]:
# Defined shared layers as global variables
repeator = RepeatVector(max_length)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [25]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attention) LSTM cell
    """
    
    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    # For grading purposes, please list 'a' first and 's_prev' second, in this order.
    concat = concatenator([a, s_prev])
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e. (≈1 lines)
    e = densor1(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. (≈1 lines)
    energies = densor2(e)
    # Use "activator" on "energies" to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(energies)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = dotor([alphas, a])
    ### END CODE HERE ###
    
    return context

In [26]:
n_a = 128 # number of units for the pre-attention, bi-directional LSTM's hidden state 'a'
n_s = 128 # number of units for the post-attention LSTM's hidden state "s"

# Please note, this is the post attention LSTM cell.  
# For the purposes of passing the automatic grader
# please do not modify this global variable.  This will be corrected once the automatic grader is also updated.
post_activation_LSTM_cell = LSTM(n_s, return_state = True) # post-attention LSTM 
output_layer = Dense(vocab_size, activation=softmax)

In [27]:
def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):

    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 (initial hidden state) and c0 (initial cell state)
    # for the decoder LSTM with shape (n_s,)
    X = Input(shape=(Tx,))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    ### START CODE HERE ###
    
    # Step 1: Define your pre-attention Bi-LSTM. (≈ 1 line)
    e = embeddingLayer(X)
    a = Bidirectional(LSTM(units = n_a ,return_sequences = True))(e)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context = one_step_attention(a, s)
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s, _, c = post_activation_LSTM_cell(inputs = context,initial_state = [s,c])
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        out = output_layer(s)
        
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model = Model(inputs = [X,s0,c0],outputs = outputs)
    
    ### END CODE HERE ###
    
    return model

In [28]:
model = model(20, 20, n_a, n_s, 20000, 20000)

In [29]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20, 200)      4000000     input_1[0][0]                    
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 128)          0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 20, 256)      336896      embedding_1[0][0]                
____________________________________________________________________________________________

In [30]:
### START CODE HERE ### (≈2 lines)
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(opt,metrics=['accuracy'],loss='sparse_categorical_crossentropy')
### END CODE HERE ###

In [31]:
s0 = np.zeros((10000, n_s))
c0 = np.zeros((10000, n_s))
outputs = list(input_labs.swapaxes(0,1))

In [32]:
model.fit([input_seq, s0, c0], outputs, epochs=5, batch_size=100)

Epoch 1/5
10000/10000 [==============================] - 159s 16ms/step - loss: 55.7823 - dense_3_loss: 1.0105 - dense_3_accuracy: 0.0039 - dense_3_accuracy_1: 0.1475 - dense_3_accuracy_2: 0.3206 - dense_3_accuracy_3: 0.4403 - dense_3_accuracy_4: 0.5501 - dense_3_accuracy_5: 0.6502 - dense_3_accuracy_6: 0.7288 - dense_3_accuracy_7: 0.7872 - dense_3_accuracy_8: 0.8321 - dense_3_accuracy_9: 0.8710 - dense_3_accuracy_10: 0.8998 - dense_3_accuracy_11: 0.9205 - dense_3_accuracy_12: 0.9367 - dense_3_accuracy_13: 0.9477 - dense_3_accuracy_14: 0.9550 - dense_3_accuracy_15: 0.9619 - dense_3_accuracy_16: 0.9675 - dense_3_accuracy_17: 0.9715 - dense_3_accuracy_18: 0.9749 - dense_3_accuracy_19: 0.9769
Epoch 2/5
10000/10000 [==============================] - 147s 15ms/step - loss: 42.6170 - dense_3_loss: 0.3521 - dense_3_accuracy: 0.0039 - dense_3_accuracy_1: 0.1492 - dense_3_accuracy_2: 0.3238 - dense_3_accuracy_3: 0.4456 - dense_3_accuracy_4: 0.5556 - dense_3_accuracy_5: 0.6571 - dense_3_accuracy

In [33]:
import math
def beam_search_decoder(predictions, top_k = 5):

    output_sequences = [([], 0)]
    
    for token_probs in predictions:
        new_sequences = []
        token_probs = token_probs.reshape(20000 ,)
        for old_seq, old_score in output_sequences:
            for char_index in range(len(token_probs)):
                new_seq = old_seq + [char_index]
                new_score = old_score + math.log(token_probs[char_index])
                new_sequences.append((new_seq, new_score))
                
        output_sequences = sorted(new_sequences, key = lambda val: val[1], reverse = True)
        output_sequences = output_sequences[:top_k]
        
    return output_sequences

In [34]:
s = [("what is your name")]
seq = tokenizer.texts_to_sequences(s)
pad = pad_sequences(seq,maxlen=max_length, truncating=trunc_type,padding = 'post')
print(pad)

[[ 15  13  18 215   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]]


In [35]:
pred = model.predict([pad , np.zeros((1, n_s)) , np.zeros((1, n_s))])

In [36]:
pred[1].shape

(1, 20000)

In [37]:
len(pred)

20

In [38]:
ans = beam_search_decoder(pred)
ans

[([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  -6.955544819619267),
 ([2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  -9.427976481762903),
 ([3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  -9.637861905639388),
 ([4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  -9.793647604894536),
 ([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  -9.85820191156175)]

In [ ]:
print(index_word[4])

i
